# Download

Downloading products is after searching one of the most important features of `eodag`. This page describes the different methods available to download products and the parameters that these methods accept.

<div class="alert alert-warning">

Warning

Downloading products from a provider whose storage is based on AWS may incur some cost.

</div>

## Setup

Results obtained from *PEPS* after a search of *Sentinel 2 Level-1C* products over France in March 2021 will be loaded in a `SearchResult`. But first, the credentials need to be set in order to be able to download anything.

In [1]:
import os
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

A workspace directory is created to store the downloaded products.

In [2]:
workspace = 'eodag_workspace_download'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

By default `eodag` saves products in the directory set by `outputs_prefix` which is by default the system temporary folder (`/tmp` on Linux) and quicklooks in a `quicklooks `subfolder of `outputs_prefix` (`tmp/quicklooks` on Linux). Here `eodag` is configured to download products in this workspace directory.

In [3]:
os.environ["EODAG__PEPS__DOWNLOAD__OUTPUTS_PREFIX"] = os.path.abspath(workspace)

Another setting that could be defined here is whether or not products need to be automatically extracted from their archive. They are extracted by default, this setting is not going to be altered here. The search result is finally loaded with `serialize_and_register`.

In [4]:
from eodag import EODataAccessGateway
dag = EODataAccessGateway()
search_results = dag.deserialize_and_register("data/download_search_results.geojson")
print(f"This SearchResult stores {len(search_results)} {search_results[0].product_type} products.")

This SearchResult stores 10 S2_MSI_L1C products.


Logging is set to see more about what `eodag` does when it downloads products.

In [46]:
from eodag import setup_logging
setup_logging(1)

In [58]:
# This code cell has a special metadata entry: "nbsphinx": "hidden"
# That hides it when the documentation is built with nbsphinx/sphinx.

# Uncomment these lines to regenerate the GeoJSON file used in this notebook.

#from eodag.api.search_result import SearchResult

#search_results = dag.search_all(
#    productType="S2_MSI_L1C",
#    start="2018-01-01",
#    end="2021-01-01",
#    geom={"lonmin": 1, "latmin": 45, "lonmax": 1.5, "latmax": 45.5},
#)
#combined_search_results = SearchResult([])
#offline_prods = [p for p in search_results if p.properties["storageStatus"] == "OFFLINE"]
#online_prods = [p for p in search_results if p.properties["storageStatus"] == "ONLINE"]
#if len(offline_prods) == 0 or len(online_prods) == 0:
#    raise ValueError("This search result must contain both ONLINE and OFFLINE products for the #notebook to be run correctly")
#combined_search_results.extend(online_prods[:5])
#combined_search_results.extend(offline_prods[:5])
#combined_search_results
#dag.serialize(combined_search_results, "data/download_search_results.geojson")

## Notebook progress bar

`eodag` displays a progress bar every time it downloads products or quicklooks. In the context of a Jupyter Notebook, an instance of the [NotebookProgressCallback](../../api_reference/utils.rst#eodag.utils.NotebookProgressCallback) class should be used instead of the default one which is more appropriate for command line usage. It should be passed to the `progress_callback` parameter of the download methods.

In [7]:
from eodag.utils import NotebookProgressCallback

## Already downloaded product

An `EOProduct` has a `location` attribute that stores either its remote location (before being downloaded) or its local location (after being downloaded). The download methods of `eodag` checks first wheter the product/quicklook to download has been already downloaded. If so, it just returns its path and does not try to download it again.

## Download quicklooks

EO products usually offer a *quicklook* image, a low resolution by-product of the original data. An `EOProduct` has a [get_quicklook](../../api_reference/eoproduct.rst#eodag.api.product._product.EOProduct.get_quicklook) method that takes care of downloading the quicklook image and returns its path. It accepts a `base_dir` (optional) parameter to specify an output directory and a `filename` (optional) parameter to specify an output filename. By default, `eodag` saves a quicklook with its ID as filename.

`get_quicklook` returns the local absolute path to the image downloaded. This is useful to precess it afterwards. Here `matplotlib` is used here to display 4 quicklooks.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig = plt.figure(figsize=(10, 8))
for i, product in enumerate(search_results, start=1):
    # This line takes care of downloading the quicklook
    quicklook_path = product.get_quicklook(progress_callback=NotebookProgressCallback())
    
    # Plot the quicklook
    img = mpimg.imread(quicklook_path)
    ax = fig.add_subplot(3, 4, i)
    ax.set_title(i - 1)
    plt.imshow(img)
plt.tight_layout()

## Download EO products

### Dynamically configure some download options

The 3 download methods introduced below accept the following optional kwargs that temporarily override the global configuration:

* `outputs_prefix` (str): absolute path to a folder where the products should be saved
* `extract` (bool): whether to automatically extract or not the downloaded product archive
* `dl_url_params` (dict): additional parameters to pass over to the download url as an url parameter

### Order OFFLINE products

As introduced in the [getting started guide](../../getting_started_guide/product_storage_status.rst) an EO product may not be available for download immediately. If the product status is `OFFLINE`, the download methods will request an order of the product and, by default, retry to download it every 2 minutes during 20 minutes. These two durations can be set with the `wait` (in minutes) and `retry` (in minutes) optional parameters of all the download methods.

The storage status of a product can be obtained from its `storageStatus` field. The status of an `OFFLINE` product is updated by `eodag` to `STAGING` when ordered and to `ONLINE` when found available.

In [8]:
[p.properties["storageStatus"] for p in search_results]

['ONLINE',
 'ONLINE',
 'ONLINE',
 'ONLINE',
 'ONLINE',
 'OFFLINE',
 'OFFLINE',
 'OFFLINE',
 'OFFLINE',
 'OFFLINE']

A `FilterProperty` can be used to filter out `OFFLINE` products to avoid triggering any product order. 

In [9]:
from eodag.plugins.crunch.filter_property import FilterProperty

In [10]:
online_search_results = search_results.crunch(
    FilterProperty(dict(storageStatus="ONLINE"))
)
[p.properties["storageStatus"] for p in online_search_results]

2021-04-13 11:16:48,768-15s eodag.plugins.crunch.filter_property [DEBUG   ] (filter_property  ) Start filtering for products matching operator.eq(product.properties['storageStatus'], ONLINE)
2021-04-13 11:16:48,771-15s eodag.plugins.crunch.filter_property [INFO    ] (filter_property  ) Finished filtering products. 5 resulting products


['ONLINE', 'ONLINE', 'ONLINE', 'ONLINE', 'ONLINE']

### Download multiple products at once

`EODataAccessGateway` offers a [download_all](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.download_all) method that takes a `SearchResult` argument and will try to download each `EOProduct` it contains. It returns a list of absolute paths to the downloaded products. For the purpose of this user guide only 2 products will be downloaded.

In [13]:
products_to_download = online_search_results[2:4]
paths = dag.download_all(products_to_download, progress_callback=NotebookProgressCallback())

2021-04-13 11:22:32,521-15s eodag.core                       [INFO    ] (core             ) Downloading 2 products


2021-04-13 11:22:32,651-15s eodag.plugins.download.base      [INFO    ] (base             ) Download url: https://peps.cnes.fr/resto/collections/S2ST/86440880-7f7d-50f0-87a7-ac1bc2bd8fbd/download


2021-04-13 11:35:12,878-15s eodag.plugins.download.http      [DEBUG   ] (http             ) Download recorded in /home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/.downloaded/a5d8e3fb7be3e74cbdd0bbe0497417ce
2021-04-13 11:35:12,887-15s eodag.plugins.download.base      [INFO    ] (base             ) Extraction activated


2021-04-13 11:35:15,337-15s eodag.plugins.download.base      [INFO    ] (base             ) Download url: https://peps.cnes.fr/resto/collections/S2ST/c51639e0-18ab-55a4-bc7c-7757a64911fa/download

2021-04-13 12:04:34,814-15s eodag.plugins.download.http      [DEBUG   ] (http             ) Download recorded in /home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/.downloaded/7ec32b9e6e6a253fa76fb278390c48d9
2021-04-13 12:04:34,823-15s eodag.plugins.download.base      [INFO    ] (base             ) Extraction activated


In [14]:
paths

['/home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/S2A_MSIL1C_20201229T110451_N0209_R094_T31TCL_20201229T131620/S2A_MSIL1C_20201229T110451_N0209_R094_T31TCL_20201229T131620.SAFE',
 '/home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/S2A_MSIL1C_20201226T105451_N0209_R051_T31TCK_20201226T130209/S2A_MSIL1C_20201226T105451_N0209_R051_T31TCK_20201226T130209.SAFE']

The command below shows that the products have been downloaded as archived files (*Sentinel* products are distributed in the [SAFE format](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/data-formats)). `eodag` then took care of extracting them to their own directory.

In [15]:
!ls {workspace}

quicklooks
S2A_MSIL1C_20201226T105451_N0209_R051_T31TCK_20201226T130209
S2A_MSIL1C_20201226T105451_N0209_R051_T31TCK_20201226T130209.zip
S2A_MSIL1C_20201229T110451_N0209_R094_T31TCL_20201229T131620
S2A_MSIL1C_20201229T110451_N0209_R094_T31TCL_20201229T131620.zip


In [21]:
!ls {paths[0]}

AUX_DATA   GRANULE  INSPIRE.xml    MTD_MSIL1C.xml
DATASTRIP  HTML     manifest.safe  rep_info


`eodag` saves a file in the `outputs_prefix` folder that contains a hash list of the products downloaded. Before downloading products, this file is read and it is checked whether the products to download were already downloaded or not. If they were, their download is skipped.

In [62]:
products_to_download = online_search_results[2:4]
dag.download_all(products_to_download, progress_callback=NotebookProgressCallback())

2021-04-13 15:27:44,258-15s eodag.core                       [INFO    ] Downloading 2 products


2021-04-13 15:27:44,335-15s eodag.plugins.download.base      [INFO    ] Download url: https://peps.cnes.fr/resto/collections/S2ST/86440880-7f7d-50f0-87a7-ac1bc2bd8fbd/download
2021-04-13 15:27:44,340-15s eodag.plugins.download.base      [INFO    ] Product already downloaded: /home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/S2A_MSIL1C_20201229T110451_N0209_R094_T31TCL_20201229T131620.zip
2021-04-13 15:27:44,343-15s eodag.plugins.download.base      [INFO    ] Extraction cancelled, destination directory already exists and is not empty: /home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/S2A_MSIL1C_20201229T110451_N0209_R094_T31TCL_20201229T131620
2021-04-13 15:27:44,347-15s eodag.plugins.download.base      [INFO    ] Download url: https://peps.cnes.fr/resto/collections/S2ST/c51639e0-18ab-55a4-bc7c-7757a64911fa/download
2021-04-13 15:27:44,352-15s eodag.plugins.download.base      [INFO    ] P

['/home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/S2A_MSIL1C_20201229T110451_N0209_R094_T31TCL_20201229T131620',
 '/home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/S2A_MSIL1C_20201226T105451_N0209_R051_T31TCK_20201226T130209']

### Download a single product

`EODataAccessGateway` also offers a [download](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.download) method that takes an `EOProduct` argument and will try to download it. It is a wrapper around the [download](../../api_reference/eoproduct.rst#eodag.api.product._product.EOProduct.download) method that each `EOProduct` has. Its advantage over the latter is that, similarly to what does `deserialize_and_register`, it adds the information required for a product to download itself if it was missing. It does so by extracting the product's provider and getting the right authentication and download plugins. Both methods return the absolute path to the downloaded product.

### Download an OFFLINE product

This examples shows what logs `eodag` emits when it tries to download a product that is originally `OFFLINE`. It is set to retry downloading the product every 1 minute after ordering it, and to stop trying (skip it if `download_all` is used) after 5 minutes.

In [51]:
offline_product = search_results.crunch(
    FilterProperty(dict(storageStatus="OFFLINE"))
)[1]
offline_product.properties["storageStatus"]

2021-04-13 14:12:55,035-15s eodag.plugins.crunch.filter_property [INFO    ] Finished filtering products. 551 resulting products


'OFFLINE'

In [52]:
path = dag.download(
    offline_product,
    wait=1,
    timeout=5,
    progress_callback=NotebookProgressCallback()
)

2021-04-13 14:13:00,963-15s eodag.plugins.download.base      [INFO    ] Download url: https://peps.cnes.fr/resto/collections/S2ST/342f1704-8931-5411-8812-d96b343e2da5/download
2021-04-13 14:13:09,814-15s eodag.plugins.download.http      [INFO    ] [Retry #1] Waiting 51s until next download try (retry every 1' for 5')


2021-04-13 14:14:07,466-15s eodag.plugins.download.http      [INFO    ] [Retry #2] Waiting 53s until next download try (retry every 1' for 5')
2021-04-13 14:15:02,877-15s eodag.plugins.download.http      [INFO    ] [Retry #3] Waiting 58s until next download try (retry every 1' for 5')


2021-04-13 14:47:04,527-15s eodag.plugins.download.base      [INFO    ] Extraction activated


2021-04-13 14:47:06,448-15s eodag.api.product                [INFO    ] Remote location of the product is still available through its 'remote_location' property: https://peps.cnes.fr/resto/collections/S2ST/342f1704-8931-5411-8812-d96b343e2da5/download




In [53]:
path

'file:///home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/S2A_MSIL1C_20201116T105331_N0209_R051_T31TCL_20201116T130215/S2A_MSIL1C_20201116T105331_N0209_R051_T31TCL_20201116T130215.SAFE'

Now the `location` attribute rightfully points to its local path.

In [57]:
offline_product.location

'file:///home/maxime/TRAVAIL/06_EODAG/01_eodag/eodag/docs/notebooks/api_user_guide/eodag_workspace_download/S2A_MSIL1C_20201116T105331_N0209_R051_T31TCL_20201116T130215/S2A_MSIL1C_20201116T105331_N0209_R051_T31TCL_20201116T130215.SAFE'